<a href="https://colab.research.google.com/github/salmaatamang1/Machine-Learning/blob/main/mnist_classification_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install tensorflow


In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_img_dir='/content/drive/MyDrive/dataset/mnist_dataset/train'
validation_img_dir='/content/drive/MyDrive/dataset/mnist_dataset/valid'

In [4]:

img_rows=224
img_cols=224

In [12]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=10, width_shift_range=0.1,
                                 height_shift_range=0.1, horizontal_flip=True, fill_mode='nearest')
validation_datagen=ImageDataGenerator(rescale=1./255,rotation_range=10, width_shift_range=0.1,
                                 height_shift_range=0.1, horizontal_flip=True, fill_mode='nearest')


In [13]:
train_generator=train_datagen.flow_from_directory(train_img_dir, target_size=(img_rows,img_cols), batch_size=20,
                                                  class_mode='categorical',shuffle=True)
validation_generator=validation_datagen.flow_from_directory(validation_img_dir,target_size=(img_rows,img_cols), batch_size=20,
                                                            class_mode='categorical',shuffle=True)

Found 10011 images belonging to 10 classes.
Found 1539 images belonging to 10 classes.


In [14]:
from keras.applications import MobileNet
MobileNet = MobileNet(weights ='imagenet', include_top = False, input_shape = (img_rows, img_cols, 3))

In [16]:
def addTopModelMobileNet(bottom_model, num_classes):
  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = Dense(512, activation='relu')(top_model)
  top_model = Dense(512, activation='relu')(top_model)
  top_model = Dense(128, activation='relu')(top_model)
  top_model = Dense(num_classes, activation='softmax')(top_model)
  return top_model



In [17]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalAveragePooling2D
from keras.models import Model

num_classes=10

fully_connected = addTopModelMobileNet(MobileNet, num_classes)
model = Model(inputs=MobileNet.input, outputs=fully_connected)
print(model.summary())


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128 

In [18]:
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])

In [19]:
history=model.fit_generator(train_generator,steps_per_epoch=10000//20, epochs=3, validation_data=validation_generator, validation_steps=1500//20)

<ipython-input-19-91e64a11a5e5>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_generator,steps_per_epoch=10000//20, epochs=3, validation_data=validation_generator, validation_steps=1500//20)


Epoch 1/3
500/500 [==============================] - 2409s 5s/step - loss: 1.5187 - accuracy: 0.8963 - val_loss: 0.4285 - val_accuracy: 0.9153
Epoch 2/3
500/500 [==============================] - 181s 363ms/step - loss: 0.0803 - accuracy: 0.9828 - val_loss: 0.1135 - val_accuracy: 0.9660
Epoch 3/3
500/500 [==============================] - 180s 359ms/step - loss: 0.0597 - accuracy: 0.9869 - val_loss: 0.1176 - val_accuracy: 0.9540


In [20]:
model.save('/content/drive/MyDrive/dataset/mnist_dataset/mnist_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
